# Week 6: Policy Simulation — Resource Allocation for Population Health

**Learning Objectives:**
- Apply cost-effectiveness analysis to real-world resource allocation decisions
- Understand diminishing returns and the cost-effectiveness frontier
- Explore the tension between efficiency and equity in health policy
- Critically evaluate the limitations of purely economic approaches to priority-setting

---

## The Challenge

You are advising a regional health authority with a **£50 million budget** for preventive health programmes over the next 5 years. Your task is to allocate this budget across available interventions to maximise population health — measured in DALYs averted.

But maximising DALYs isn't the only consideration. You'll also need to think about:
- **Equity**: Do the benefits reach those in greatest need?
- **Uncertainty**: How confident are we in these estimates?
- **Time horizon**: Should we value immediate gains over long-term prevention?
- **Political feasibility**: Some effective interventions face public resistance

---

## 1. Setup

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib plotly ipywidgets -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntSlider, FloatSlider, Dropdown, Button, Output, VBox, HBox, Label, HTML
from IPython.display import display, clear_output, Markdown
import warnings
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)

## 2. The Interventions

Below are 8 interventions you can fund. The cost-effectiveness estimates are derived from published literature, NICE guidance, and PHE analyses. All figures are illustrative but grounded in real evidence.

### Key Parameters

- **Base cost per DALY**: Cost-effectiveness at initial scale
- **Maximum capacity**: How much can be spent before saturation
- **Diminishing returns**: How quickly cost-effectiveness worsens at scale
- **Equity impact**: Distribution of benefits across deprivation quintiles
- **Time to impact**: When health gains materialise

In [ ]:
# Define interventions with realistic parameters
interventions = {
    'salt_reduction': {
        'name': 'Salt Reduction Campaign',
        'description': 'Population-wide campaign: reformulation agreements with industry, public awareness, labelling',
        'base_cost_per_daly': 1500,  # Very cost-effective (WHO Best Buy)
        'max_capacity_millions': 15,  # £15M maximum useful spend
        'diminishing_factor': 1.8,    # Moderate diminishing returns
        'equity_distribution': [0.25, 0.23, 0.20, 0.17, 0.15],  # Slightly pro-poor (diet quality gradient)
        'time_to_impact_years': 2,
        'uncertainty_range': 0.3,     # ±30% uncertainty
        'evidence_quality': 'High',
        'category': 'Population'
    },
    'folic_acid': {
        'name': 'Folic Acid Fortification',
        'description': 'Mandatory fortification of flour to prevent neural tube defects',
        'base_cost_per_daly': 2000,
        'max_capacity_millions': 5,   # Limited scope - one-off policy
        'diminishing_factor': 1.2,    # Minimal diminishing returns (binary policy)
        'equity_distribution': [0.22, 0.21, 0.20, 0.19, 0.18],  # Roughly equal
        'time_to_impact_years': 1,
        'uncertainty_range': 0.2,
        'evidence_quality': 'High',
        'category': 'Population'
    },
    'weight_management': {
        'name': 'Adult Weight Management',
        'description': 'Structured programmes: behavioural support, dietary counselling, physical activity',
        'base_cost_per_daly': 8000,
        'max_capacity_millions': 25,
        'diminishing_factor': 2.5,    # High diminishing returns (hard to reach)
        'equity_distribution': [0.15, 0.18, 0.22, 0.23, 0.22],  # Tends to reach affluent
        'time_to_impact_years': 3,
        'uncertainty_range': 0.4,
        'evidence_quality': 'Moderate',
        'category': 'Individual'
    },
    'diabetes_prevention': {
        'name': 'NHS Diabetes Prevention Programme',
        'description': 'Intensive lifestyle intervention for people with pre-diabetes',
        'base_cost_per_daly': 6000,
        'max_capacity_millions': 20,
        'diminishing_factor': 2.0,
        'equity_distribution': [0.18, 0.20, 0.21, 0.21, 0.20],  # Moderate equity
        'time_to_impact_years': 4,
        'uncertainty_range': 0.35,
        'evidence_quality': 'High',
        'category': 'Targeted'
    },
    'school_meals': {
        'name': 'School Food Standards Enhancement',
        'description': 'Improved nutritional standards, free school meal expansion, food education',
        'base_cost_per_daly': 12000,  # Looks expensive but...
        'max_capacity_millions': 30,
        'diminishing_factor': 1.5,
        'equity_distribution': [0.30, 0.25, 0.20, 0.15, 0.10],  # Strongly pro-poor
        'time_to_impact_years': 15,   # Long lag to adult health outcomes
        'uncertainty_range': 0.5,
        'evidence_quality': 'Moderate',
        'category': 'Population'
    },
    'smoking_cessation': {
        'name': 'Smoking Cessation Services',
        'description': 'NHS stop smoking services with pharmacotherapy',
        'base_cost_per_daly': 3500,
        'max_capacity_millions': 20,
        'diminishing_factor': 2.2,
        'equity_distribution': [0.28, 0.24, 0.20, 0.16, 0.12],  # Pro-poor (smoking gradient)
        'time_to_impact_years': 5,
        'uncertainty_range': 0.25,
        'evidence_quality': 'High',
        'category': 'Individual'
    },
    'hypertension_screening': {
        'name': 'Hypertension Detection & Treatment',
        'description': 'Community screening, GP follow-up, medication adherence support',
        'base_cost_per_daly': 5000,
        'max_capacity_millions': 25,
        'diminishing_factor': 1.8,
        'equity_distribution': [0.20, 0.20, 0.20, 0.20, 0.20],  # Equal if well-designed
        'time_to_impact_years': 3,
        'uncertainty_range': 0.2,
        'evidence_quality': 'High',
        'category': 'Clinical'
    },
    'sdil_extension': {
        'name': 'Sugar Tax Extension',
        'description': 'Extend SDIL to confectionery and other high-sugar products',
        'base_cost_per_daly': 800,    # Very cost-effective (revenue generating)
        'max_capacity_millions': 8,   # Implementation costs only
        'diminishing_factor': 1.3,
        'equity_distribution': [0.24, 0.23, 0.20, 0.18, 0.15],  # Mildly pro-poor
        'time_to_impact_years': 3,
        'uncertainty_range': 0.45,    # High uncertainty (industry response unknown)
        'evidence_quality': 'Moderate',
        'category': 'Population'
    }
}

# Create summary dataframe
summary_data = []
for key, v in interventions.items():
    summary_data.append({
        'Intervention': v['name'],
        'Category': v['category'],
        'Base £/DALY': f"£{v['base_cost_per_daly']:,}",
        'Max Budget (£M)': v['max_capacity_millions'],
        'Time to Impact': f"{v['time_to_impact_years']} years",
        'Evidence': v['evidence_quality'],
        'Uncertainty': f"±{v['uncertainty_range']*100:.0f}%"
    })

summary_df = pd.DataFrame(summary_data)
print("Available Interventions")
print("="*100)
display(summary_df)

In [ ]:
# Display detailed descriptions
print("\nIntervention Details")
print("="*100)
for key, v in interventions.items():
    print(f"\n{v['name'].upper()}")
    print(f"  {v['description']}")
    print(f"  → Equity profile: Q1 (most deprived) receives {v['equity_distribution'][0]*100:.0f}% of benefits")

## 3. The Cost-Effectiveness Model

### Diminishing Returns

Real interventions don't scale linearly. The first £1M reaches the most accessible population with the clearest need. Subsequent spending encounters:
- Harder-to-reach populations
- Implementation challenges
- Saturation effects

We model this with a power function:

$$\text{Cost per DALY at spend } x = \text{Base cost} \times \left(\frac{x}{\text{first unit}}\right)^{(\alpha - 1)}$$

Where $\alpha$ is the diminishing returns factor (higher = steeper diminishing returns).

In [ ]:
def calculate_dalys_averted(spend_millions, intervention):
    """
    Calculate DALYs averted for a given spend, accounting for diminishing returns.
    
    Uses numerical integration over spend increments.
    """
    if spend_millions <= 0:
        return 0
    
    # Cap at maximum capacity
    spend_millions = min(spend_millions, intervention['max_capacity_millions'])
    
    base_cost = intervention['base_cost_per_daly']
    alpha = intervention['diminishing_factor']
    
    # Integrate in £0.1M increments
    increments = np.arange(0.1, spend_millions + 0.1, 0.1)
    total_dalys = 0
    
    for i, x in enumerate(increments):
        # Cost per DALY at this level of spend
        marginal_cost = base_cost * (x ** (alpha - 1))
        # DALYs from this £0.1M increment
        dalys_increment = (0.1 * 1_000_000) / marginal_cost
        total_dalys += dalys_increment
    
    return total_dalys


def calculate_marginal_cost_per_daly(spend_millions, intervention):
    """
    Calculate the marginal cost per DALY at a given spend level.
    """
    if spend_millions <= 0:
        return intervention['base_cost_per_daly']
    
    base_cost = intervention['base_cost_per_daly']
    alpha = intervention['diminishing_factor']
    
    return base_cost * (spend_millions ** (alpha - 1))


def calculate_equity_dalys(total_dalys, equity_distribution):
    """
    Distribute DALYs across deprivation quintiles.
    Returns dict with Q1 (most deprived) to Q5 (least deprived).
    """
    return {
        f'Q{i+1}': total_dalys * equity_distribution[i]
        for i in range(5)
    }

In [ ]:
# Visualise diminishing returns for each intervention
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, (key, intervention) in enumerate(interventions.items()):
    ax = axes[idx]
    
    spends = np.linspace(0.1, intervention['max_capacity_millions'], 50)
    marginal_costs = [calculate_marginal_cost_per_daly(s, intervention) for s in spends]
    
    ax.plot(spends, marginal_costs, 'b-', linewidth=2)
    ax.axhline(y=20000, color='r', linestyle='--', alpha=0.7, label='NICE threshold (£20k)')
    ax.axhline(y=30000, color='orange', linestyle='--', alpha=0.7, label='NICE upper (£30k)')
    
    ax.set_xlabel('Spend (£M)')
    ax.set_ylabel('Marginal £/DALY')
    ax.set_title(intervention['name'], fontsize=10)
    ax.set_ylim(0, min(50000, max(marginal_costs) * 1.1))
    
plt.suptitle('Marginal Cost-Effectiveness by Spending Level\n(How cost per DALY changes as you spend more)', 
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. The Budget Allocation Game

Now it's your turn. You have **£50 million** to allocate. Use the sliders below to set your budget for each intervention.

The dashboard will update in real-time to show:
- Total DALYs averted
- Budget remaining
- Distribution of health gains by intervention
- Equity impact across deprivation quintiles

In [ ]:
# Create the interactive budget allocation interface

TOTAL_BUDGET = 50  # £50 million

# Create sliders for each intervention
sliders = {}
slider_widgets = []

for key, intervention in interventions.items():
    slider = IntSlider(
        value=0,
        min=0,
        max=intervention['max_capacity_millions'],
        step=1,
        description='',
        continuous_update=True,
        readout=True,
        readout_format='d',
        layout=widgets.Layout(width='250px')
    )
    sliders[key] = slider
    
    label = widgets.HTML(
        value=f"<b>{intervention['name']}</b><br><small>Max: £{intervention['max_capacity_millions']}M</small>",
        layout=widgets.Layout(width='220px')
    )
    
    unit_label = widgets.HTML(value="£M", layout=widgets.Layout(width='30px'))
    
    slider_widgets.append(HBox([label, slider, unit_label]))

# Budget display
budget_html = widgets.HTML(value="")

# Output areas
results_output = Output()
charts_output = Output()

def update_dashboard(change=None):
    """Update the dashboard when sliders change."""
    
    # Calculate total spend
    allocations = {key: slider.value for key, slider in sliders.items()}
    total_spend = sum(allocations.values())
    remaining = TOTAL_BUDGET - total_spend
    
    # Update budget display
    if remaining < 0:
        budget_html.value = f"""
        <div style='padding: 10px; background-color: #ffcccc; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: red; margin: 0;'>⚠️ OVER BUDGET by £{-remaining}M</h3>
            <p style='margin: 5px 0;'>Total allocated: £{total_spend}M / £{TOTAL_BUDGET}M</p>
        </div>
        """
    elif remaining == 0:
        budget_html.value = f"""
        <div style='padding: 10px; background-color: #ccffcc; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: green; margin: 0;'>✓ Budget Fully Allocated</h3>
            <p style='margin: 5px 0;'>Total allocated: £{total_spend}M / £{TOTAL_BUDGET}M</p>
        </div>
        """
    else:
        budget_html.value = f"""
        <div style='padding: 10px; background-color: #ffffcc; border-radius: 5px; margin: 10px 0;'>
            <h3 style='margin: 0;'>Budget Remaining: £{remaining}M</h3>
            <p style='margin: 5px 0;'>Total allocated: £{total_spend}M / £{TOTAL_BUDGET}M</p>
        </div>
        """
    
    # Calculate results
    results = []
    equity_totals = {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0}
    
    for key, spend in allocations.items():
        intervention = interventions[key]
        dalys = calculate_dalys_averted(spend, intervention)
        
        # Calculate equity distribution
        equity_dalys = calculate_equity_dalys(dalys, intervention['equity_distribution'])
        for q, val in equity_dalys.items():
            equity_totals[q] += val
        
        if spend > 0:
            avg_cost = (spend * 1_000_000) / dalys if dalys > 0 else 0
            marginal_cost = calculate_marginal_cost_per_daly(spend, intervention)
        else:
            avg_cost = 0
            marginal_cost = intervention['base_cost_per_daly']
        
        results.append({
            'intervention': intervention['name'],
            'key': key,
            'spend': spend,
            'dalys': dalys,
            'avg_cost': avg_cost,
            'marginal_cost': marginal_cost,
            'time_to_impact': intervention['time_to_impact_years'],
            'category': intervention['category']
        })
    
    total_dalys = sum(r['dalys'] for r in results)
    
    # Update results display
    with results_output:
        clear_output(wait=True)
        
        print("="*80)
        print(f"TOTAL DALYs AVERTED: {total_dalys:,.0f}")
        if total_spend > 0:
            print(f"OVERALL COST-EFFECTIVENESS: £{(total_spend * 1_000_000) / total_dalys:,.0f} per DALY")
        print("="*80)
        
        # Results table
        results_df = pd.DataFrame(results)
        results_df = results_df[results_df['spend'] > 0].sort_values('dalys', ascending=False)
        
        if len(results_df) > 0:
            display_df = results_df[['intervention', 'spend', 'dalys', 'avg_cost', 'marginal_cost']].copy()
            display_df.columns = ['Intervention', 'Spend (£M)', 'DALYs Averted', 'Avg £/DALY', 'Marginal £/DALY']
            display(display_df.style.format({
                'Spend (£M)': '£{:.0f}M',
                'DALYs Averted': '{:,.0f}',
                'Avg £/DALY': '£{:,.0f}',
                'Marginal £/DALY': '£{:,.0f}'
            }).hide(axis='index'))
    
    # Update charts
    with charts_output:
        clear_output(wait=True)
        
        if total_dalys > 0:
            fig = make_subplots(
                rows=1, cols=3,
                subplot_titles=('DALYs Averted by Intervention', 
                               'Equity: DALYs by Deprivation Quintile',
                               'Time to Health Impact'),
                specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]]
            )
            
            # Chart 1: DALYs by intervention
            funded = [r for r in results if r['spend'] > 0]
            if funded:
                funded_sorted = sorted(funded, key=lambda x: x['dalys'], reverse=True)
                fig.add_trace(
                    go.Bar(
                        x=[r['intervention'] for r in funded_sorted],
                        y=[r['dalys'] for r in funded_sorted],
                        marker_color='steelblue',
                        text=[f"{r['dalys']:,.0f}" for r in funded_sorted],
                        textposition='outside'
                    ),
                    row=1, col=1
                )
            
            # Chart 2: Equity distribution
            quintile_labels = ['Q1\n(Most Deprived)', 'Q2', 'Q3', 'Q4', 'Q5\n(Least Deprived)']
            quintile_values = [equity_totals[f'Q{i+1}'] for i in range(5)]
            colors = ['#d73027', '#fc8d59', '#fee090', '#91bfdb', '#4575b4']
            
            fig.add_trace(
                go.Bar(
                    x=quintile_labels,
                    y=quintile_values,
                    marker_color=colors,
                    text=[f"{v:,.0f}" for v in quintile_values],
                    textposition='outside'
                ),
                row=1, col=2
            )
            
            # Chart 3: Time to impact
            # Group DALYs by time to impact
            time_groups = {}
            for r in results:
                t = r['time_to_impact']
                time_groups[t] = time_groups.get(t, 0) + r['dalys']
            
            times = sorted(time_groups.keys())
            time_dalys = [time_groups[t] for t in times]
            
            fig.add_trace(
                go.Bar(
                    x=[f"{t} years" for t in times],
                    y=time_dalys,
                    marker_color='coral',
                    text=[f"{v:,.0f}" for v in time_dalys],
                    textposition='outside'
                ),
                row=1, col=3
            )
            
            fig.update_layout(
                height=400,
                showlegend=False,
                title_text="Your Allocation Results"
            )
            fig.update_xaxes(tickangle=45, row=1, col=1)
            
            fig.show()
            
            # Equity summary
            q1_share = equity_totals['Q1'] / total_dalys * 100 if total_dalys > 0 else 0
            q5_share = equity_totals['Q5'] / total_dalys * 100 if total_dalys > 0 else 0
            
            print("\n" + "="*80)
            print("EQUITY SUMMARY")
            print("="*80)
            print(f"Share of benefits to most deprived quintile (Q1): {q1_share:.1f}%")
            print(f"Share of benefits to least deprived quintile (Q5): {q5_share:.1f}%")
            
            if q1_share > 22:
                print("→ Your allocation is PRO-EQUITY (benefits skewed toward deprived areas)")
            elif q1_share < 18:
                print("→ Your allocation may WIDEN INEQUALITIES (benefits skewed toward affluent areas)")
            else:
                print("→ Your allocation has NEUTRAL equity impact")

# Connect sliders to update function
for slider in sliders.values():
    slider.observe(update_dashboard, names='value')

# Layout
print("BUDGET ALLOCATION GAME")
print("Allocate your £50M budget using the sliders below:")
print("")

display(VBox([
    budget_html,
    VBox(slider_widgets),
    results_output,
    charts_output
]))

# Initial update
update_dashboard()

## 5. Strategy Comparison

Let's compare three pre-defined strategies to see how different priorities lead to different outcomes.

In [ ]:
# Define three contrasting strategies
strategies = {
    'Maximum Efficiency': {
        'description': 'Allocate purely based on cost-effectiveness, ignoring equity',
        'allocations': {
            'salt_reduction': 10,
            'folic_acid': 5,
            'sdil_extension': 8,
            'smoking_cessation': 12,
            'hypertension_screening': 15,
            'diabetes_prevention': 0,
            'weight_management': 0,
            'school_meals': 0
        }
    },
    'Equity Focus': {
        'description': 'Prioritise interventions that benefit deprived populations',
        'allocations': {
            'salt_reduction': 8,
            'folic_acid': 5,
            'sdil_extension': 5,
            'smoking_cessation': 15,
            'hypertension_screening': 0,
            'diabetes_prevention': 0,
            'weight_management': 0,
            'school_meals': 17
        }
    },
    'Balanced Portfolio': {
        'description': 'Diversified approach across intervention types',
        'allocations': {
            'salt_reduction': 8,
            'folic_acid': 5,
            'sdil_extension': 5,
            'smoking_cessation': 8,
            'hypertension_screening': 10,
            'diabetes_prevention': 7,
            'weight_management': 0,
            'school_meals': 7
        }
    }
}

def evaluate_strategy(allocations):
    """Evaluate a strategy and return key metrics."""
    total_dalys = 0
    equity_totals = {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0}
    total_spend = sum(allocations.values())
    
    for key, spend in allocations.items():
        intervention = interventions[key]
        dalys = calculate_dalys_averted(spend, intervention)
        total_dalys += dalys
        
        equity_dalys = calculate_equity_dalys(dalys, intervention['equity_distribution'])
        for q, val in equity_dalys.items():
            equity_totals[q] += val
    
    return {
        'total_dalys': total_dalys,
        'total_spend': total_spend,
        'cost_per_daly': (total_spend * 1_000_000) / total_dalys if total_dalys > 0 else 0,
        'q1_share': equity_totals['Q1'] / total_dalys * 100 if total_dalys > 0 else 0,
        'equity_totals': equity_totals
    }

# Evaluate all strategies
print("STRATEGY COMPARISON")
print("="*100)

comparison_results = []
for name, strategy in strategies.items():
    metrics = evaluate_strategy(strategy['allocations'])
    comparison_results.append({
        'Strategy': name,
        'Description': strategy['description'],
        'Total DALYs': metrics['total_dalys'],
        'Cost/DALY': metrics['cost_per_daly'],
        'Q1 Share (%)': metrics['q1_share']
    })
    print(f"\n{name.upper()}")
    print(f"  {strategy['description']}")
    print(f"  DALYs averted: {metrics['total_dalys']:,.0f}")
    print(f"  Cost per DALY: £{metrics['cost_per_daly']:,.0f}")
    print(f"  Share to most deprived (Q1): {metrics['q1_share']:.1f}%")

comparison_df = pd.DataFrame(comparison_results)

In [ ]:
# Visualise strategy comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Total DALYs Averted', 'Equity: Share to Most Deprived (Q1)')
)

colors = ['#2ecc71', '#3498db', '#9b59b6']

fig.add_trace(
    go.Bar(
        x=comparison_df['Strategy'],
        y=comparison_df['Total DALYs'],
        marker_color=colors,
        text=[f"{v:,.0f}" for v in comparison_df['Total DALYs']],
        textposition='outside'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=comparison_df['Strategy'],
        y=comparison_df['Q1 Share (%)'],
        marker_color=colors,
        text=[f"{v:.1f}%" for v in comparison_df['Q1 Share (%)']],
        textposition='outside'
    ),
    row=1, col=2
)

# Add reference line for equal distribution
fig.add_hline(y=20, line_dash="dash", line_color="red", 
              annotation_text="Equal distribution (20%)", row=1, col=2)

fig.update_layout(
    height=400,
    showlegend=False,
    title_text="Strategy Comparison: Efficiency vs Equity Trade-off"
)

fig.show()

## 6. The Efficiency-Equity Frontier

There's a fundamental tension in health policy:
- **Efficiency**: Maximise total health gains per pound spent
- **Equity**: Prioritise those in greatest need, even if less cost-effective

Let's visualise this trade-off.

In [ ]:
# Generate many random allocations to map the frontier
np.random.seed(42)
n_simulations = 500

simulation_results = []

for _ in range(n_simulations):
    # Generate random allocation
    raw_weights = np.random.dirichlet(np.ones(8)) * 50
    
    allocations = {}
    for i, key in enumerate(interventions.keys()):
        max_cap = interventions[key]['max_capacity_millions']
        allocations[key] = min(raw_weights[i], max_cap)
    
    # Scale to use full budget
    total = sum(allocations.values())
    if total > 0:
        scale = 50 / total
        allocations = {k: min(v * scale, interventions[k]['max_capacity_millions']) 
                      for k, v in allocations.items()}
    
    metrics = evaluate_strategy(allocations)
    simulation_results.append({
        'dalys': metrics['total_dalys'],
        'q1_share': metrics['q1_share']
    })

# Add our named strategies
for name, strategy in strategies.items():
    metrics = evaluate_strategy(strategy['allocations'])
    simulation_results.append({
        'dalys': metrics['total_dalys'],
        'q1_share': metrics['q1_share'],
        'name': name
    })

sim_df = pd.DataFrame(simulation_results)

# Plot
fig = go.Figure()

# Random allocations
unnamed = sim_df[sim_df['name'].isna()] if 'name' in sim_df.columns else sim_df
fig.add_trace(go.Scatter(
    x=unnamed['dalys'],
    y=unnamed['q1_share'],
    mode='markers',
    marker=dict(size=5, color='lightgray', opacity=0.5),
    name='Random allocations'
))

# Named strategies
for name, color in zip(['Maximum Efficiency', 'Equity Focus', 'Balanced Portfolio'],
                       ['#2ecc71', '#3498db', '#9b59b6']):
    strategy_point = sim_df[sim_df.get('name') == name]
    if len(strategy_point) > 0:
        fig.add_trace(go.Scatter(
            x=strategy_point['dalys'],
            y=strategy_point['q1_share'],
            mode='markers+text',
            marker=dict(size=15, color=color),
            text=[name],
            textposition='top center',
            name=name
        ))

fig.add_hline(y=20, line_dash="dash", line_color="red", 
              annotation_text="Equal distribution")

fig.update_layout(
    title='The Efficiency-Equity Trade-off',
    xaxis_title='Total DALYs Averted (Efficiency →)',
    yaxis_title='Share to Most Deprived Q1 (Equity →)',
    height=500,
    showlegend=True
)

fig.show()

print("\nThe cloud of grey points shows possible allocations.")
print("Points in the upper-right are best: high DALYs AND pro-equity.")
print("But notice the trade-off: it's hard to maximise both simultaneously.")

## 7. Incorporating Uncertainty

All our estimates have uncertainty. Let's see how this affects our confidence in different strategies.

In [ ]:
def evaluate_strategy_with_uncertainty(allocations, n_samples=1000):
    """
    Evaluate strategy using Monte Carlo simulation to capture uncertainty.
    """
    results = []
    
    for _ in range(n_samples):
        total_dalys = 0
        
        for key, spend in allocations.items():
            if spend <= 0:
                continue
                
            intervention = interventions[key]
            
            # Sample from uncertainty range
            uncertainty = intervention['uncertainty_range']
            multiplier = np.random.uniform(1 - uncertainty, 1 + uncertainty)
            
            dalys = calculate_dalys_averted(spend, intervention) * multiplier
            total_dalys += dalys
        
        results.append(total_dalys)
    
    return np.array(results)

# Evaluate each strategy with uncertainty
uncertainty_results = {}

for name, strategy in strategies.items():
    samples = evaluate_strategy_with_uncertainty(strategy['allocations'])
    uncertainty_results[name] = {
        'samples': samples,
        'mean': np.mean(samples),
        'p5': np.percentile(samples, 5),
        'p95': np.percentile(samples, 95)
    }

In [ ]:
# Visualise uncertainty
fig = go.Figure()

colors = ['#2ecc71', '#3498db', '#9b59b6']

for i, (name, results) in enumerate(uncertainty_results.items()):
    fig.add_trace(go.Violin(
        y=results['samples'],
        name=name,
        box_visible=True,
        meanline_visible=True,
        fillcolor=colors[i],
        line_color='black',
        opacity=0.7
    ))

fig.update_layout(
    title='DALYs Averted with Uncertainty (Monte Carlo Simulation)',
    yaxis_title='DALYs Averted',
    height=500,
    showlegend=False
)

fig.show()

print("\nUncertainty Summary (90% confidence interval):")
print("="*60)
for name, results in uncertainty_results.items():
    print(f"{name}:")
    print(f"  Mean: {results['mean']:,.0f} DALYs")
    print(f"  90% CI: [{results['p5']:,.0f} - {results['p95']:,.0f}]")
    print()

## 8. Equity Weighting

Some argue that a DALY averted in a deprived area should count for more than one in an affluent area. This is **equity weighting**.

Let's see how different equity weights change the optimal allocation.

In [ ]:
def evaluate_with_equity_weights(allocations, equity_weights):
    """
    Evaluate strategy with equity-weighted DALYs.
    
    equity_weights: dict mapping Q1-Q5 to weight multipliers
    """
    total_weighted_dalys = 0
    
    for key, spend in allocations.items():
        intervention = interventions[key]
        dalys = calculate_dalys_averted(spend, intervention)
        
        equity_dalys = calculate_equity_dalys(dalys, intervention['equity_distribution'])
        
        for q, val in equity_dalys.items():
            total_weighted_dalys += val * equity_weights[q]
    
    return total_weighted_dalys

# Define equity weight scenarios
weight_scenarios = {
    'No weighting (all equal)': {'Q1': 1.0, 'Q2': 1.0, 'Q3': 1.0, 'Q4': 1.0, 'Q5': 1.0},
    'Mild pro-equity (1.5x for Q1)': {'Q1': 1.5, 'Q2': 1.25, 'Q3': 1.0, 'Q4': 0.9, 'Q5': 0.8},
    'Strong pro-equity (2x for Q1)': {'Q1': 2.0, 'Q2': 1.5, 'Q3': 1.0, 'Q4': 0.75, 'Q5': 0.5},
}

print("Impact of Equity Weighting on Strategy Rankings")
print("="*80)

for scenario_name, weights in weight_scenarios.items():
    print(f"\n{scenario_name}")
    print("-"*50)
    
    results = []
    for strategy_name, strategy in strategies.items():
        weighted_dalys = evaluate_with_equity_weights(strategy['allocations'], weights)
        results.append((strategy_name, weighted_dalys))
    
    results.sort(key=lambda x: x[1], reverse=True)
    
    for rank, (name, dalys) in enumerate(results, 1):
        print(f"  {rank}. {name}: {dalys:,.0f} weighted DALYs")

In [ ]:
# Interactive equity weight explorer
equity_slider = FloatSlider(
    value=1.0,
    min=1.0,
    max=3.0,
    step=0.1,
    description='Q1 Weight:',
    continuous_update=True,
    style={'description_width': 'initial'}
)

equity_output = Output()

def update_equity_analysis(change):
    q1_weight = equity_slider.value
    
    # Linear interpolation for other quintiles
    weights = {
        'Q1': q1_weight,
        'Q2': 1 + (q1_weight - 1) * 0.6,
        'Q3': 1.0,
        'Q4': 1 - (q1_weight - 1) * 0.3,
        'Q5': 1 - (q1_weight - 1) * 0.5
    }
    
    with equity_output:
        clear_output(wait=True)
        
        print(f"Equity weights: Q1={weights['Q1']:.1f}, Q2={weights['Q2']:.1f}, "
              f"Q3={weights['Q3']:.1f}, Q4={weights['Q4']:.1f}, Q5={weights['Q5']:.1f}")
        print("\nStrategy rankings with these weights:")
        
        results = []
        for name, strategy in strategies.items():
            weighted = evaluate_with_equity_weights(strategy['allocations'], weights)
            unweighted = evaluate_strategy(strategy['allocations'])['total_dalys']
            results.append({
                'Strategy': name,
                'Unweighted DALYs': unweighted,
                'Weighted DALYs': weighted,
                'Change': (weighted / unweighted - 1) * 100
            })
        
        df = pd.DataFrame(results).sort_values('Weighted DALYs', ascending=False)
        display(df.style.format({
            'Unweighted DALYs': '{:,.0f}',
            'Weighted DALYs': '{:,.0f}',
            'Change': '{:+.1f}%'
        }).hide(axis='index'))

equity_slider.observe(update_equity_analysis, names='value')

print("\nExplore how equity weighting changes strategy rankings:")
print("(Slide to increase the weight given to health gains in deprived areas)\n")
display(equity_slider)
display(equity_output)

update_equity_analysis(None)

## 9. Discussion Questions

### Reflection on Your Allocation

1. **What drove your choices?** Did you prioritise cost-effectiveness, equity, evidence quality, or something else?

2. **Did you fund school meals?** It looks expensive (£12k/DALY) but has strong equity benefits. Should we discount children's future health gains?

3. **How did you handle uncertainty?** Did you avoid interventions with wide confidence intervals, or accept risk for potentially large gains?

### Broader Questions

4. **Should we use equity weights?** If a DALY in Middlesbrough counts for twice as much as one in Surrey, is that fair? Who decides?

5. **What's missing from this model?**
   - Political feasibility (sugar tax extension is controversial)
   - Implementation capacity
   - Public preferences
   - Spillover effects (children's health affects parents)

6. **Is cost-effectiveness analysis the right framework?** What are its blind spots?

### For Further Thought

7. The **school meals** intervention looks expensive on a 5-year horizon but potentially transformative over 30 years. How should we handle interventions with very long time horizons in democratic systems where governments change every 4-5 years?

8. **Proportionate universalism** suggests targeting resources within universal systems. How might you redesign these interventions to be both efficient AND equitable?

## 10. Build Your Own Strategy

Use the cell below to define and evaluate your own named strategy.

In [ ]:
# Define your strategy here
my_strategy = {
    'salt_reduction': 0,      # £M allocated
    'folic_acid': 0,
    'weight_management': 0,
    'diabetes_prevention': 0,
    'school_meals': 0,
    'smoking_cessation': 0,
    'hypertension_screening': 0,
    'sdil_extension': 0
}

# Check budget
total = sum(my_strategy.values())
print(f"Total allocated: £{total}M / £50M")

if total > 50:
    print("⚠️ Over budget!")
elif total < 50:
    print(f"💰 £{50-total}M remaining")
else:
    print("✓ Budget fully allocated")

# Evaluate
if total > 0:
    metrics = evaluate_strategy(my_strategy)
    print(f"\nResults:")
    print(f"  DALYs averted: {metrics['total_dalys']:,.0f}")
    print(f"  Cost per DALY: £{metrics['cost_per_daly']:,.0f}")
    print(f"  Share to Q1 (most deprived): {metrics['q1_share']:.1f}%")

## 11. Key Takeaways

1. **Cost-effectiveness varies by scale**: The first pound buys more health than the tenth million

2. **Efficiency and equity often trade off**: The most cost-effective interventions may not reach those in greatest need

3. **Uncertainty matters**: Confidence in estimates should influence allocation decisions

4. **Time horizon shapes priorities**: Interventions with long-term benefits may look poor over short horizons

5. **Values are embedded in methods**: Discount rates, equity weights, and included costs all reflect value choices

6. **Models simplify reality**: Political feasibility, implementation capacity, and public preferences all matter

---

## References

- NICE (2013). Guide to the methods of technology appraisal.
- WHO (2017). Tackling NCDs: Best buys and other recommended interventions.
- Cookson R et al. (2017). Using cost-effectiveness analysis to address health equity concerns. *Value in Health*.
- Briggs A et al. (2006). Decision Modelling for Health Economic Evaluation. Oxford University Press.
- PHE (2018). Health matters: preventing cardiovascular disease.